研究者への資本配分(を想定していると思われる)パートの再現

In [1]:
using DataFrames
using PyPlot

In [2]:
function initialize(N, C0, average_talent, talent_distribution)
    t = [average_talent for _ = 1:N] + talent_distribution*randn(N)
    for i = 1:N
        if t[i] < 0.0
            t[i] = 0.0
        elseif t[i] > 1.0
            t[i] = 1.0
        end
    end

    init_info_dic = Dict(
        "1:name" => [i for i = 1:N],
        "2:talent" => t,
        "3:meet ruck counter" => [0 for _ = 1:N],
        "4:get ruck counter" => [0 for _ = 1:N],
        "5:unruck counter" => [0 for _ = 1:N],
        "6:capital" => [C0 for _ = 1:N],
        "7:x" => rand(N),
        "8:y" => rand(N),
        "9:r2" => [0.0 for _ = 1:N],
        "A:talented" => [false for _ = 1:N],
    )

    df = DataFrame(init_info_dic)
    sort!(df, [:"2:talent"], rev=true)
    for i = 1:N
        if df[i,2] > average_talent + talent_distribution
            df[i,10] = true
        else
            break
        end
    end

    hist(df[:,2], bins=Int(floor(sqrt(N))))
    print("histgram of the talent")
    xlabel("talent")
    return df
end

initialize (generic function with 1 method)

In [3]:
function run(df, cross_range, N_E, I)
    ruck_place = rand(N_E,2)
    unruck_place = rand(N_E,2)
    
    I = size(df)[1]
    for t = 1:I
        for p = 1:size(ruck_place)[1]
            x, y = ruck_place[p,:]
            for i = 1:size(df)[1]
                df[i,9] = 0
                df[i,9] += (df[i,7] - x)^2
                df[i,9] += (df[i,8] - y)^2
            
                if df[i,9] <= cross_range^2
                    df[i,3] += 1
                    if rand() <= df[i,2]
                        df[i,4] += 1
                        df[i,6] *= 2
                    end
                end
            end
        end
        for p = 1:size(unruck_place)[1]
            x, y = unruck_place[p,:]
            for i = 1:size(df)[1]
                df[i,9] = 0
                df[i,9] += (df[i,7] - x)^2
                df[i,9] += (df[i,8] - y)^2
            
                if df[i,9] <= cross_range^2
                    df[i,5] += 1
                    df[i,6] /= 2
                end
            end
        end
         
        ruck_place += 0.011*randn(N_E, 2)
        unruck_place += 0.011*randn(N_E, 2)
        for i = 1:size(ruck_place)[1]
            while ruck_place[i] < 0
                ruck_place[i] += 1
            end
            while ruck_place[i] > 1
                ruck_place[i] -= 1
            end
        end
        for i = 1:size(unruck_place)[1]
            while unruck_place[i] < 0
                unruck_place[i] += 1
            end
            while unruck_place[i] > 1
                unruck_place[i] -= 1
            end
        end
    end
end

run (generic function with 1 method)

In [4]:
function cal_P_T(df, C0)
    P_T, c, d, e = 0, 0, 0, 0
    for i = 1:size(df)[1]
        if df[i,10]
            c += 1
            if df[i,6] > C0
                d += 1
            elseif df[i,6] == C0
                e += 1
            end
        end
    end
    f = d + e*d/(c-e)
    P_T = 100*f/size(df)[1]
    return P_T
end

cal_P_T (generic function with 1 method)

In [ ]:
N = 10^3
I = 80
C0 = 10.0
N_E = Int(N // 2)
cross_range = 0.005
average_talent = 0.6
talent_distribution = 0.1

TIMES = 100
P_T0 = 0
for t = 1:TIMES
    df = initialize(N, C0, average_talent, talent_distribution)
    run(df, cross_range, N_E, I)
    P_T0 += cal_P_T(df, C0)
    println(P_T0, t)
end
P_T0 /= TIMES
P_T0